In [14]:
# Import stuff

import re
from BCBio import GFF
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation



In [78]:
# Read files

file_tsv = open("../run_dros_genome/full_table_dros.tsv", 'r')


# Extract BUSCO IDs, start and end from table of hits into SeqRecord object, each BUSCO as a SeqFeature

hit_rec = SeqRecord(seq='', id='hits')
for line in file_tsv.readlines():
    hit = (re.search(r'(\S*)\sComplete\s(\S*)\s(\S*)\s(\S*)\s\S*', line))
    if hit:
        hit_rec.features.append(SeqFeature(FeatureLocation(int(hit.group(3)), int(hit.group(4))), id=hit.group(1), type='gene', qualifiers={'contig': hit.group(2)}))
        
                    
print(hit_rec)

#file_tsv.close()

ID: hits
Name: <unknown name>
Description: <unknown description>
Number of features: 264
''


In [88]:

# Match the BUSCOs to augustus predicted genes in gff file
gff_recs = []
correct_genes = SeqRecord(seq='', id='correct_genes')
limit_info = dict(
        gff_type = ["gene"]) # Only want genes


for busco in hit_rec.features:
    filename = busco.id # gff filenames are {busco_id}.out.1
    i = 1
    while True:
        try:
            file_gff = open("../run_dros_genome/augustus_output/predicted_genes/"+filename+".out."+str(i))
            for record in GFF.parse(file_gff, limit_info=limit_info):
                gff_recs.append(record)
            i += 1
        except: # read all files for that BUSCO
            break
        

# Find augustus predicted genes from the gff that match BUSCOs
for rec in gff_recs:
    for hit in hit_rec.features:
        for feature in rec.features:
            if hit.location.start-1 == feature.location.start and hit.location.end == feature.location.end: # For some reason start has 1 nt diff...
                correct_genes.features.append(feature)
                break


print(correct_genes)
#file_gff.close()

# Is the BUSCO id needed for further analysis?  Don't think so

ID: correct_genes
Name: <unknown name>
Description: <unknown description>
Number of features: 279
''
